# Data Summary and Goal

## Summary

Tanzania, as a developing country, struggles with providing clean water to its population of over 57,000,000. There are many waterpoints already established in the country, but some are in need of repair while others have failed altogether.

## Goal

Build a classifier to predict the condition of a water well, using information provided in the data. This information includes:
- Date
- Location
- Source
- Funder
- And more!

This data is from the DrivenData.org website. It is part of the "Pump It Up: Data Mining the Water Table" dfetition. DrivenData decided to split the data up into two sets, the "Training Set" and the "Test Set". 

It is implied by the names that we are to use the training set for creating our models, and the test set to test them. For this project, we considered merging the two dataframes in order to have more data to work with, however there are 59,400 entries in the training set and therefore more than enough to make good predicitons. 

If our models are subpar, we may merge the tables to aquire more data points to potentially improve model efficacy.

# Data Cleaning and Feature Engineering

## Import Libraries and Data

In [3]:
# Import Pandas
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# ignore all future warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

# Load data into  Pandas dataframes
status_groups = pd.read_csv('Data/status_groups.csv')
testset = pd.read_csv('Data/test_set.csv')
df = pd.read_csv('Data/training_set.csv')


# Let's add our target series to the dataframe!
status_groups.drop(['id'], axis=1, inplace=True)
df = pd.concat([df, status_groups], axis=1)

# Analyze shape of dataset
print(f'Shape of dataset: {df.shape}')
display(df.head())


Shape of dataset: (59400, 41)


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


## EDA

In [ ]:
df.columns

LATITUDE AND LONGITUDE-----------------------------------------------------------------------------------------------------------------

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
sns.scatterplot(x='latitude', y='longitude', hue='status_group',data=df)
plt.title('Scatterplot visualizing the distribution of well statuses around the map of Tanzania', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/map.png')

These points are represetnative of the map of Tanzania if it was rotated roughly 90 degrees clockwise. Using this information, we can see that there is a large cluster of non-functional wells to the West and South West, a large cluster of functional wells to the North and South East

PERMIT ------------------------------------------------------------------------------------------------------------------------

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
sns.countplot(x='permit', hue='status_group', data=df)
plt.title('Permit Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/permit_count.png')

Wells with permits have a larger functional to non-functional ration than those without.

REGION ------------------------------------------------------------------------------------------------------------------------

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.countplot(x='region', hue='status_group', data=df)
plt.title('Region Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/region.png')

Region can be a good indicator of well status, because how many functional vs non-functional wells differ a lot from region to region. For example, if you were to pick a well at random in the region 'Iringa', you would most likely find a functional well, but if you picked a well at random in 'Mtwara', you would most likely find a non-functional well!

BASIN ------------------------------------------------------------------------------------------------------------------------

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.countplot(x='basin', hue='status_group', data=df)
plt.title('Basin Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/basin.png')

There are some basins that are clearly more successful when establishing wells. Two regions that stick out are 'Ruvuma/Southern Coast' and 'Lake Rukwa', because both of those basins seem to have a majority non-functional wells!

*** Since we are keeping basin, region, and latitude and longitude, we do not need any more columns that provide location data or it would become to noisy. 

CONSTRUCTION YEAR -------------------------------------------------------------------------------------------------------------

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
sns.countplot(x='construction_year', hue='status_group', data=df)
plt.title('Construction Year Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/construction year.png')

When ignoring the data of contruction_year==0, we can clearly see that newer wells are functional more often, while older wells are non-functional more often.

MANAGEMENT ------------------------------------------------------------------------------------------------------------------------

In [ ]:
display(df.management.unique())
display(df.management_group.unique())

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.countplot(x='management', hue='status_group', data=df)
plt.title('Management Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/management.png')

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.countplot(x='management_group', hue='status_group', data=df)
plt.title('Management Group Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/management_group.png')

Both columns provide the same data however 'management' is more detailed, so we will keep that column and drop the other.

PAYMENT ------------------------------------------------------------------------------------------------------------------------

In [ ]:
display(df.payment.unique())
display(df.payment_type.unique())

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.countplot(x='payment', hue='status_group', data=df)
plt.title('Payment Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/payment.png')

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.countplot(x='payment_type', hue='status_group', data=df)
plt.title('Payment Type Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/payment_type.png')

'payment' and 'payment_type' are columns with identical data. We can drop one. We will drop 'payment'

QUANTITY ------------------------------------------------------------------------------------------------------------------------

In [ ]:
display(df.quantity.unique())
display(df.quantity_group.unique())

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.countplot(x='quantity', hue='status_group', data=df)
plt.title('Quantity Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/quantity.png')

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.countplot(x='quantity_group', hue='status_group', data=df)
plt.title('Quantity Group Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/quantity_group.png')

'quantity_group' and 'quantity' are identical columns. We can drop one, and we will drop 'quantity_group' because 'qauntity' is a title that is understood easier.

WATERPOINT TYPE ------------------------------------------------------------------------------------------------------------------------

In [ ]:
display(df.waterpoint_type.unique())
display(df.waterpoint_type_group.unique())

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.countplot(x='waterpoint_type_group', hue='status_group', data=df)
plt.title('WaterPoint Type Group Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/waterpoint_type_group.png')

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.countplot(x='waterpoint_type', hue='status_group', data=df)
plt.title('WaterPoint Type Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/waterpoint_type.png')

The two 'waterpoint_type' columns display redundant information. 'waterpoint_type' is more desireable because there are very noticeable difference between the two groups that 'waterpoint_type_group' joined; 'communal standpoint' and 'communal standpoint multiple'. The majority of wells with a waterpoint type of 'communal standpoint multiple' were non-functional, while the majority of wells with a waterpoint type of 'communal standpoint' were functional. This is an important difference we wish to express.

Overall, handpumps and communal standpoints (and to a lesser extent, improved spring) show a majority occurence of functional wells, while the rest show a majority occurence of non-functional wells

EXTRACTION TYPE ------------------------------------------------------------------------------------------------------------------------

In [ ]:
display(df['extraction_type_class'].unique())
display(df['extraction_type'].unique())
display(df['extraction_type_group'].unique())

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
sns.countplot(x='extraction_type_class', hue='status_group', data=df)
plt.title('Extraction Type Class Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/extraction_type_class.png')

The three 'extraction_type' columns display the same information, however both 'extraction_type' and 'extraction_type_group' have many more unique values than 'extraction_type_class'. When looking at the columns, it is clear that 'extraction_type_class' is more interpretable, and reduces the cardinality of this 'extraction_type' data considerably, without reducing its effectiveness in conveying the information it intends to. For this reason, we will be keeping 'extraction_type_class' and dropping the others.

Overall, the data of wells that use a gravity and handpump extraction type have the highest occurence of functional wells, while motorpump and other extraction type are most likely to be non-functional in comparison. Submersible also has a majority functional, however it is not as pronounced as handpump and gravity.

SOURCE  ------------------------------------------------------------------------------------------------------------------------

In [ ]:
display(df['source_class'].unique())
display(df['source'].unique())
display(df['source_type'].unique())

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
sns.countplot(x='source', hue='status_group', data=df)
plt.title('Source Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/source.png')

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
sns.countplot(x='source_type', hue='status_group', data=df)
plt.title('Source Type Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/source_type.png')

The three 'source' columns provide the same information, with 'source_class' being the least specific and 'source' being the most specific. 'Source' provides mroe detailed information without severely increasing cardinality, and therefore we will keep that column and drop the others.

Springs, rainwater harvesting, shallow wells, rivers, DBH machines, and hand boreholes (hand dtw) are most likely to be functional in comparison to the other source types. Noticeably, wells with their source as a river are functional significantly more often than if the source is a lake. This information was not conveyed in the 'source_type' column.

QUALITY ------------------------------------------------------------------------------------------------------------------------

In [ ]:
display(df.water_quality.unique())
display(df.quality_group.unique())

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
sns.countplot(x='water_quality', hue='status_group', data=df)
plt.title('Water Quality Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/water_quality.png')

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
sns.countplot(x='quality_group', hue='status_group', data=df)
plt.title('Quanlity Group Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/quanlity_group.png')

The 'water_quality' and 'quality_group' columns provide redundant information. 'quality_group' is more desireable than 'quality_group' even though the latter is more specific because the latter's specificity does not add any meaninful information to our analysis.

Water of 'good' quality results in mostly functional wells, while those classified as an 'unknown' quality are mostly non-functional. They are unknown potentially due to the well being out of commission for such a long time that the water quality was no longer known by the time of recording.

AMOUNT_TSH -----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
df.amount_tsh.value_counts().sum()

In [ ]:
df.amount_tsh.value_counts()

In [ ]:
print(f'Percentage of data that is zero: {41639/59400}')

The amount_tsh column describes the 'Total Static Head' of each well, which is defined as “The vertical height of a stationary column of liquid produced by a pump, measured from the suction level". This information could be useful for clasification, however 70% of all the values given are 0. We are unable to determine if the TSH level of these wells are actually zero, or if the number was simply used as a placeholder to describe a lack of information or a specific TSH status. Due to this, we will be dropping the column.

## Drop Columns and Deal with Missing Values

In [ ]:
df.isna().sum()[df.isna().sum()>0]

In [ ]:
# Construction_year
df.date_recorded = pd.to_datetime(df.date_recorded)
df.construction_year = np.where(df.construction_year==0, df.date_recorded.dt.year, df.construction_year)

In [ ]:
# Label encode the target variable
status_labels = {'status_group':{'non functional': 0, 'functional': 1, 'functional needs repair': 2}}
df = df.replace(status_labels)
df.status_group.value_counts()


# Drop redundant and unneeded columns
to_drop = ['scheme_name', 'recorded_by', 'wpt_name', 'extraction_type', 'extraction_type_group',
           'region_code', 'district_code', 'lga', 'ward', 'public_meeting', 'date_recorded', 
           'source_type', 'source_class', 'waterpoint_type', 'water_quality', 'management_group', 
           'payment', 'quantity_group','subvillage', 'num_private', 'scheme_management', 'amount_tsh', 'latitude', 'longitude']


# Deal with missing values
df.drop(to_drop, axis=1, inplace=True)
df.permit.fillna(False, inplace=True)
df.dropna(axis=0, inplace=True)

# Set 'id' as the index of the dataframe
df.set_index('id', inplace=True)

In [ ]:
df.columns, df.shape

## Grouping and Labeling Column Values

In [ ]:
# Funder
df.funder.replace(to_replace='0', value='unknown', inplace=True)
df.funder.value_counts().head(20)

In [ ]:
other = list(df.funder[df['funder'].map(df['funder'].value_counts()) < 484].values)
other
df['funder'].replace(other, 'other', inplace=True)

df.funder.replace(to_replace='0', value='Unknown', inplace=True)


In [ ]:
# Installer

other = list(df.installer[df['installer'].map(df['installer'].value_counts()) < 392].values)
other
df['installer'].replace(other, 'other', inplace=True)

In [ ]:
# Population

df['population'].replace(to_replace = 0 , value =df.population.mean(), inplace=True)
# Areas usually don't have zero population,so we are going to replace those values with the mean!

In [ ]:
# Permit

df.permit.replace({True:1, False:0}, inplace=True)

In [ ]:
def population(obs):
    s=''
    x=obs['population']
    if(0<x<=100):
        s='Less than 100'
    elif(100<x<=200):
        s='Between 100 and 200'
    elif(200<x<=300):
        s='Between 200 and 300'
    elif(300<x<=400):
        s='between 300 and 400'
    elif(400<x<=500):
        s='between 400 and 500'
    elif(500<x):
        s='Over 500'
    elif(x==0):
        s='No population'
    return s
df['population']=df.apply(population,axis=1)


In [ ]:
# # Amount_tsh
# # Bin
# conditions = [df.amount_tsh==0,(df.amount_tsh>0)&(df.amount_tsh<=10),(df.amount_tsh>10)&(df.amount_tsh<=100), (df.amount_tsh>100)&(df.amount_tsh<=1000),
#              (df.amount_tsh>1000)&(df.amount_tsh<=2000), (df.amount_tsh>2000)&(df.amount_tsh<=10000), (df.amount_tsh>10000)&(df.amount_tsh<=100000),
#              df.amount_tsh>100000]
# choices = ['zero', '1 to 10', '11 to 100', '101 to 1k', '1k to 2k', '2k to 10k', '10k to 100k', 'greater than 100k']
# df['amount_tsh'] = np.select(conditions, choices)

In [ ]:
df.gps_height = pd.qcut(df.gps_height, 8, duplicates='drop', 
        labels=['-90m - sea level', 'sea level to 46m', '46m to 393m', '393m to 1017m', '1017m to 1316m', '1316m to 1586.75m', '1586.75m to 2770m'])

After grouping all the columns, including all the numeric columns, we ended up with 16 columns, a pretty good amount of columns for data analysis, reducing noise but not too much.

# More visualizations

In [ ]:
df1 = df.loc[df['funder']== 'Government Of Tanzania']
df2 = df.loc[df['funder']== 'Tasaf']              
df3 = df.loc[df['funder']== 'Danida'] 
df4 = df.loc[df['funder']== 'Hesawa'] 
df5 = df.loc[df['funder']== 'Rwssp'] 
df6 = df.loc[df['funder']== 'World Bank'] 
df7 = df.loc[df['funder']== 'Kkkt'] 
df8 = df.loc[df['funder']== 'World Vision']
df9 = df.loc[df['funder']== 'Unicef'] 
df10 = df.loc[df['funder']== 'unknown'] 
df11 = df.loc[df['funder']== 'District Council'] 
df12 = df.loc[df['funder']== 'Dhv'] 
df13 = df.loc[df['funder']== 'Private Individual'] 
df14 = df.loc[df['funder']== 'Dwsp'] 
df15 = df.loc[df['funder']== 'Norad'] 
df16 = df.loc[df['funder']== 'Germany Republi']
df17 = df.loc[df['funder']== 'Tcrs']
df18 = df.loc[df['funder']== 'Ministry Of Water']
df19 = df.loc[df['funder']== 'Water']
df20 = df.loc[df['funder']== 'Dwe']

top_20 = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,
                    df13,df14,df15,df16,df17,df18,df19,df20], ignore_index=True)

fig, ax = plt.subplots(figsize=(20,20))
sns.countplot(x='funder', hue="status_group", data=top_20)
plt.title('Top 20 Funder Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/funder_top_20.png')

Who funded the well has a considerable effect on the functionality of it. The Government of Tanzania, Hesawa, World Bank, Tcrs, Ministry of Water (ironically), and DWE have funded a majority of wells that were non-functional at the time this data was recorded. Overall, it seems as if private individuals and the Germany Republic funded wells that had the highest ratio of functional to non-functional at the time this data was recorded.

In [ ]:
fig, ax = plt.subplots(figsize=(25,15))
sns.countplot(x='population', hue='status_group', data=df)
plt.title('Population Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/population.png')

The lower the population, the more wells are present, most likely due to the fact that Tanzania is still composed of villages and hamlets with low urbanization. The less people around a well, the higher the likelihood of the well being functional. This could be attributed to the possibility that overuse results in non-functionality. 

Latitude and longitude data is displayed above. There are certain areas that seem to have more functional or non-functional wells. This could provide useful classification for our model based on specific location.

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
sns.countplot(x='gps_height', hue='status_group',data=df)
plt.title('GPS Height Count', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/gps_height.png')

Locations at or below sea level seem to have a higher number of functional wells. 

NEED TO USE GEOPANDAS TO UNDERSTAND THIS BETTER

# Model Building

In [ ]:
df.status_group.value_counts().plot(kind='bar', color='orange')
plt.title('Class Distribution', fontsize = 20)
plt.tight_layout()
fig.savefig('./images/class_distribution.png')

There is a clear class imbalance for our target variable. We will have to deal with this during our modeling.


Potential methods:
- SMOTE
- Undersampling
- Ensemble methods
- Cost-sensitive algorithms

## Import Libraries/Modules, Define Variables, OneHotEncode

In [81]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
import category_encoders as ce
from sklearn.compose import ColumnTransformer

# plot_confusion_matrix is a handy visual tool, added in the latest version of scikit-learn
# if you are running an older version, comment out this line and just use confusion_matrix

from sklearn.metrics import plot_confusion_matrix, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report

In [82]:
# Identify features and target
features = df.drop('status_group', axis=1)
target = df.status_group

# Dummy the features
features_dummied = pd.get_dummies(features, drop_first=True)
features_dummied.head()

,permit,construction_year,funder_Dhv,funder_District Council,funder_Dwe,funder_Dwsp,funder_Germany Republi,funder_Government Of Tanzania,funder_Hesawa,funder_Kkkt,...,source_rainwater harvesting,source_river,source_shallow well,source_spring,source_unknown,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other
id,,,,,,,,,,,,,,,,,,,,,
69572,0,1999,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
8776,1,2010,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
34310,1,2009,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
67743,1,1986,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
19728,1,2011,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0


When scoring our model, we are less focused on the precision of the model as a whole, but rather on the specific scores for recall and precision depending on class. While we still want higher precision for the model, we believe that the evaluation of these class specific metrics are more important because they help predict the specific problem in a more practical manner. 

For class 0, non-functional, recall is the most important metric. Recall is used when there is a high cost associated with a false negative. In this case, a false negative would indicate that we predicted that a truly non-functional well actually needs repair, or is completely functional. If we assume that these predictions are being made in order to find out where to allocate resources to bring water to an area, predicting that a well works or just needs repair could result in not enough resources being allocated to the area, if any at all. This could effectively deprive an area of a functioning well!

For class 1, functional, precision is the most important metric. Precision is used when there is a high cost associated with a false positive. In this case, a false positive would mean that we predict that a well that is truly non-functional or needs repair is functional. If we assume once again that these predicitons are being evaluated to make a decision on where to allocate resources, this could be detrimental because we may decide not to allocate resources to a well that actually needs it because we think it is fully functional when it actually isn't! If we have low precision for class 1, we could be leaving many areas without functional wells without even knowing it.

For class 2, needs repair, recall is the most important metric. In this case a false negative would indicate that we predicted that a well that actually needs repair is functional or not-functional. It would not be a big deal to falsely predict that a well needs repair, because if resources are allocated to that area and they find that the well actually works, then hooray, they don't need to use those resources. It also would not be a big deal to falsely predict that the well needs repair when it actually is non-functional, because when the area is surveyed, this will hopefully be noticed and the necessary resources will be allocated! 
However, it is detrimental to predict that a well is functional when it actually needs repair, because then we would be depriving the area of water, because there is little reason to spend the time to survey the area if it is predicted that the well is functional. It is also detrimental to predict that a well is non-functional if it actually needs repair, because then too many resources may be allocated (this is assuming that it takes less resources to repair a well than to build it from scratch). So false negatives are very costly here, and therefore recall is most important



### Create pipeline

In [83]:
from sklearn.pipeline import Pipeline
def machine_learn(model):
    model_pipeline = Pipeline([('ss', StandardScaler()), ('model', model)])
    fitted_model = model_pipeline.fit(X_train, y_train)
    print("Accuracy Score:", fitted_model.score(X_test, y_test))
    preds = fitted_model.predict(X_test)
    print(classification_report(y_test, preds))

### Split data and SMOTE

In [84]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features_dummied, target, test_size=0.2, random_state=123)

# SMOTE
X_resampled, y_resampled = SMOTE(sampling_strategy='minority',random_state = 123).fit_sample(X_train, y_train)

## Decision Tree

In [85]:
# Build model
tree = DecisionTreeClassifier(random_state=123)

machine_learn(tree)

Accuracy Score: 0.7646108268246701
              precision    recall  f1-score   support

           0       0.78      0.76      0.77      4362
           1       0.79      0.83      0.81      6023
           2       0.39      0.29      0.33       754

    accuracy                           0.76     11139
   macro avg       0.65      0.63      0.64     11139
weighted avg       0.76      0.76      0.76     11139



## Random Forest

In [86]:
# Build model
forest = RandomForestClassifier(random_state=123, verbose=1)

machine_learn(forest)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    7.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy Score: 0.7860669719005297
              precision    recall  f1-score   support

           0       0.81      0.76      0.79      4362
           1       0.79      0.87      0.83      6023
           2       0.48      0.28      0.36       754

    accuracy                           0.79     11139
   macro avg       0.69      0.64      0.66     11139
weighted avg       0.78      0.79      0.78     11139



[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished


In [87]:
parameters_forest = {
    'max_depth': [15, 20, 25] ,
    'n_estimators': [150, 200, 250, 300],
    'criterion': ['gini', 'entropy']}

In [88]:
grid_search_forest = GridSearchCV(
    estimator=forest,
    param_grid=parameters_forest,
    n_jobs = -1,
    cv = 5,
    verbose=True
)

In [89]:
grid_search_forest.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  7.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:   27.4s finished


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(random_state=123, verbose=1),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [15, 20, 25],
                         'n_estimators': [150, 200, 250, 300]},
             verbose=True)

In [90]:
grid_search_forest.best_params_

{'criterion': 'entropy', 'max_depth': 20, 'n_estimators': 250}

In [91]:
grid_search_forest.best_score_

0.7914170697360013

In [92]:
# Rebuild model
forest = RandomForestClassifier(criterion='entropy', max_depth=15, n_estimators=150, random_state=123, verbose=1)

machine_learn(forest)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    6.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy Score: 0.7795134213125056
              precision    recall  f1-score   support

           0       0.86      0.68      0.76      4362
           1       0.74      0.94      0.83      6023
           2       0.75      0.12      0.21       754

    accuracy                           0.78     11139
   macro avg       0.79      0.58      0.60     11139
weighted avg       0.79      0.78      0.76     11139



[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    0.2s finished


## XGBoost

In [93]:
xgb = XGBClassifier(learning_rate=0.05, max_depth=35, random_state=123, objective = 'multi:softprob', num_class=3, verbosity=1)
machine_learn(xgb)

Accuracy Score: 0.7905557051799982
              precision    recall  f1-score   support

           0       0.82      0.76      0.79      4362
           1       0.79      0.88      0.83      6023
           2       0.51      0.29      0.37       754

    accuracy                           0.79     11139
   macro avg       0.71      0.64      0.66     11139
weighted avg       0.78      0.79      0.78     11139



In [94]:
parameters_xgb = {
    'max_depth': [12, 16, 24] ,
    'n_estimators': [160, 220, 250],
    'learning_rate': [0.12, 0.001, 0.01, 0.05]}

In [95]:
grid_search_xgb = GridSearchCV(
    estimator=xgb,
    param_grid=parameters_xgb,
    n_jobs = 10,
    cv = 5,
    verbose=True
)

In [96]:
grid_search_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed: 41.9min


KeyboardInterrupt: 

In [ ]:
grid_search.best_params_

## Baseline

In [78]:
df.status_group.value_counts()

1    30226
0    21589
2     3877
Name: status_group, dtype: int64

In [80]:
df.status_group.value_counts()[1]/df.status_group.value_counts().sum()

0.5427350427350427

## Analysis

When first evaluating all three models with default settings and a random seed of 123, we found that DecisionTree performed the least favorably, XGBoost performed the most favorably, and RandomForest sits comfortably in the middle. Using this information we decided to focus on RandomForest and XGBoost exclusively for our GridSearch. Using the parameter grids described above, we narrowed our hyperparameters down to specific values for each model:

RandomForest: (criterion='entropy', max_depth=15, n_estimators=150, random_state=123)

XGBoost: (learning_rate=0.05, max_depth=35, random_state=123, objective = 'multi:softprob', num_class=3)

Overall, XGBoost remained our most favorable model, with recall for class 0 and 2 being the highest out of all the models while also trying to maximize precision for class 1. The model as a whole had an accuracy of 79%, the highest accuracy out of any of our models.


# Conclusion

In conclusion, the best model based on our research is XGBoost. We chose this based on accuracy score, recall score for classses 0 and 2, and precision score for class 1. Our overall accuracy was 79%, significantly higher than our baseline accuracy of 54.3%!

For further plans, it would be beneficial to continue to run gridsearches and more models in order to try and maximize our scores of interest. We would need more time for this due to the computation time of running the models, as well as to account for the potential feature engineering and data manipulation that could be necessary depending on the models' results.